# Site App with Database 

* About Me Section
* Notepad: Create, Read
* Counter

## Create a database server using Docker

### Postgres Docker

In [ ]:
# Without volume/data connected
!docker run --name postgres-sql -e POSTGRES_USER=matt -e POSTGRES_PASSWORD=password  -d postgres

In [ ]:
# With volume/data connected
!docker run -d \
    --name postgres-sql \
    -e POSTGRES_USER=matt \
    -e POSTGRES_PASSWORD=password \
    -v $(pwd)/data/:/var/lib/postgresql/data/ \
    -p 5000:5432 \
    postgres

In [ ]:
!docker ps

In [ ]:
!ls -a

In [ ]:
# Check the data
# In a terminal
# sudo su
# cd $(pwd)/data/
# ls -a

In [ ]:
# Test the data of the DB
# docker rm -f <CONTAINER_ID>
# run the docker run <...>
# check the pgAdmin
# the data persists

### pgAdmin

In [ ]:
!docker run -p 8000:80 \
    -e 'PGADMIN_DEFAULT_EMAIL=matt@example.com' \
    -e 'PGADMIN_DEFAULT_PASSWORD=password' \
    -d dpage/pgadmin4

In [ ]:
!docker ps

### Create a firewall (GCP)

In [ ]:
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:5000,tcp:8000,tcp:5401,tcp:9000 --source-ranges=0.0.0.0/0

### Open pgAdmin Browser

In [ ]:
# http://<IP_ADDRESS>:<PORT>
# Login with credentials

# Register Server
# HOST: <IP_ADDRESS>
# PORT: 5000
# USER: <POSTGRES_USER>
# PASSWORD: <PASSWORD>

## Install and import libraries

In [ ]:
!pip install streamlit psycopg2-binary -q

In [1]:
import streamlit as st
import psycopg2

In [ ]:
# Restart the terminal

In [4]:
%%writefile env.py
# DB Credentials
DBNAME="matt"
USER='matt'
HOST='35.193.80.15'
PORT='5000'
PASSWORD='password'

Overwriting env.py


In [7]:
%%writefile app.py
# Import libraries
import streamlit as st
import psycopg2
import os
import time
from env import *

# Header
st.write("# Notepad :notebook:")
st.caption("""
            Add your thoughts here! It will be stored in a database! \n
            :warning: :red[Do not add sensitive data].
            """)
st.subheader("",divider="rainbow")

# Variable
database_name = DBNAME

con = psycopg2.connect(f"""
                       dbname={DBNAME}
                       user={USER}
                       host={HOST}
                       port={PORT}
                       password={PASSWORD}
                       """)
cur = con.cursor()
# Create a table if not exists
cur.execute("CREATE TABLE IF NOT EXISTS notes(id serial PRIMARY KEY, name varchar, header varchar, note varchar, time varchar)")
con.commit()

# Inputs
name = st.text_input("Your Name")
header = st.text_input("Header")
note = st.text_area("Note",)
if st.button("Add a note"):
    time = time.strftime("Date: %Y-%m-%d | Time: %H:%M:%S UTC")
    st.write(f""" \n
            ### :pencil: {header} \n
            {note}
            :man: {name} \n
            :watch: {time}""")
    st.success("Successful Added.")
    # st.balloons()
    ### Insert into adatabase
    SQL = "INSERT INTO notes (name, header, note, time) VALUES(%s, %s, %s, %s);"
    data = (name, header, note, time)
    cur.execute(SQL, data)
    con.commit()
    
# Previous Notes 
st.subheader("",divider="rainbow")
st.write("### *Previous Notes*")
# Write the data
cur.execute("""
            SELECT * 
            FROM notes
            ORDER BY time DESC
            """)
for name, header, note, time in cur.fetchall():
    st.write(f""" \n
            ### :pencil: {header} \n
            {note}
            :man: {name} \n
            :watch: {time}""")
    st.subheader("",divider="gray")
    
# Close Connection
cur.close()
con.close()


Overwriting app.py


In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit psycopg2-binary -q
COPY app.py .
EXPOSE 8501
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0"]

In [ ]:
# Build an image
!docker build -t notepad-app . -q

In [ ]:
# List images
!docker images

In [ ]:
# Run the app in detached mode
!docker run -d -p 8501:8501 -v $(pwd):/app notepad-app

In [ ]:
!docker ps -a

In [ ]:
# Remove all docker ps
# !docker rm -f $(docker ps -aq)